In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import contextily as ctx

import sys, os
import sklearn
import datetime

import importlib
import itertools
from tqdm import tqdm
import matplotlib.pyplot as plt

from sklearn.neighbors import KDTree, BallTree

In [2]:
import st_toolkit as geohl
importlib.reload(geohl)

import cri_calc as cri
importlib.reload(cri)

import cri_helper as helper
importlib.reload(helper)

<module 'nn_evaluation' from '/Users/andrewt/Documents/Univ. Piraeus Research Center/VCRA/nn_evaluation.py'>

## Load Models
  * ### Pick the best performing models (w.r.t RMSL[og]E) per Fold

In [3]:
vcra_skf_results_df = pd.read_pickle('./data/pickle/vcra_skf_results_df_v14.pickle')
best_models = vcra_skf_results_df.groupby(level=0, sort=False).rmsle.idxmin()

In [4]:
models = vcra_skf_results_df.loc[best_models].copy()

In [5]:
models

,,instance,y_true,y_pred,acc,mae,rmse,rmsle,r2_score
model,,,,,,,,,
cart_vcra,2,"(StandardScaler(), (DecisionTreeRegressor(max_...","[0.9386086811364309, 0.8531393744585727, 0.925...","[0.9386849135864637, 0.8546624509781502, 0.924...",0.855561,0.039232,0.115009,0.084237,0.855561
svm_vcra,3,"(StandardScaler(), SVR(gamma='auto', verbose=T...","[0.9421287794930369, 0.9303150848403041, 0.937...","[1.0, 0.9274339036683672, 1.0, 0.9634113052502...",0.581359,0.118443,0.194147,0.143246,0.588275
rvm_vcra,0,"(StandardScaler(), EMRVR(init_alpha=1.57582934...","[0.9386086811364309, 0.8939913491298078, 0.853...","[0.9730571392498675, 0.7508551282157878, 0.881...",0.575050,0.127419,0.192971,0.137354,0.593421
mlp_vcra,0,"(StandardScaler(), MLPRegressor(early_stopping...","[0.9751822620069324, 0.7722869884296601, 0.0, ...","[0.9430493146245351, 0.7587157368294114, 0.0, ...",0.964867,0.022683,0.055928,0.040701,0.965836


## Check inference time on the busiest (in terms of concurrent encounters) timeslice of the Norway Dataset

In [6]:
gdf_vcra = pd.read_pickle('./data/norway-dataset/oslo_jan_mar_2019_4w_prep_encountering.vcra_dataset_v14.pickle')

In [7]:
timestamp_max_traffic = gdf_vcra.groupby(level=0).apply(len).idxmax()
timeslice_max_traffic = gdf_vcra.xs((timestamp_max_traffic,), level=0)

In [9]:
timestamp_max_traffic

Timestamp('2019-01-09 10:26:30')

  * ### CART-VCRA (Li et al.)

In [21]:
# cart_vcra_features = ['dist_euclid', 'own_speed', 'target_speed', 'own_course', 'target_course', 'azimuth_angle_target_to_own']
cart_vcra_features = ['dist_euclid', 'own_speed', 'target_speed', 'own_course_rad', 'target_course_rad', 'azimuth_angle_target_to_own']

cart_vcra_samples = timeslice_max_traffic.loc[:, cart_vcra_features].copy()
cart_vcra_model = models.loc['cart_vcra'].instance.values[0]

In [11]:
# cart_vcra_model.set_params(**{'randomforestregressor__n_jobs':-1})
cart_vcra_model.set_params(**{'randomforestregressor__n_jobs':1})

cart_vcra_model.set_params(**{'randomforestregressor__verbose':False})

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestregressor',
                 RandomForestRegressor(n_jobs=1, random_state=10,
                                       verbose=False))])

In [12]:
%%timeit -n 1000
cart_vcra_model.predict(cart_vcra_samples.values)

3.38 ms ± 56 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


  * ### SVM-VCRA (Gang et al.)

In [13]:
# svm_vcra_features = ['dist_euclid', 'own_speed', 'target_speed', 'own_course', 'target_course', 'relative_bearing_target_to_own']
svm_vcra_features = ['dist_euclid', 'own_speed', 'target_speed', 'own_course_rad', 'target_course_rad', 'relative_bearing_target_to_own']

svm_vcra_samples = timeslice_max_traffic.loc[:, svm_vcra_features].copy()
svm_vcra_model = models.loc['svm_vcra'].instance.values[0]

In [14]:
%%timeit -n 100
svm_vcra_model.predict(svm_vcra_samples.values)

43.4 ms ± 430 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


  * ### RVM-VCRA (Park et al.)

In [15]:
rvm_vcra_features = ['dist_euclid', 'own_speed', 'target_speed', 'own_course', 'target_course', 'relative_bearing_target_to_own', 'own_length', 'target_length']
rvm_vcra_samples = timeslice_max_traffic.loc[:, rvm_vcra_features].copy()
rvm_vcra_model = models.loc['rvm_vcra'].instance.values[0]

In [16]:
%%timeit -n 100
rvm_vcra_model.predict(rvm_vcra_samples.values)

287 µs ± 45.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


  * ### MLP-VCRA (Ours)

In [17]:
mlp_vcra_features = ['own_speed', 'own_course', 'target_speed', 'target_course', 'dist_euclid', 'azimuth_angle_target_to_own', 'rel_movement_direction']
mlp_vcra_samples = timeslice_max_traffic.loc[:, mlp_vcra_features].copy()
mlp_vcra_model = models.loc['mlp_vcra'].instance.values[0]

In [18]:
%%timeit -n 100
mlp_vcra_model.predict(mlp_vcra_samples.values)

219 µs ± 110 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


  * ### "Fuzzy" Formulae

In [19]:
def calculate_cri_(ves_pair):
    own = dict(
        course_rad=helper.degrees_to_radians(ves_pair.own_course),
        length_nmi=ves_pair.own_length / 1852,
        speed=ves_pair.own_speed
    )
    
    target = dict(
        course_rad=helper.degrees_to_radians(ves_pair.target_course),
        length_nmi=ves_pair.target_length / 1852,
        speed=ves_pair.target_speed
    )

    dist_euclid, speed_r, rel_movement_direction, azimuth_angle_target_to_own, relative_bearing_target_to_own, dcpa, tcpa = ves_pair.dist_euclid, \
    ves_pair.speed_r, ves_pair.rel_movement_direction, ves_pair.azimuth_angle_target_to_own, \
    ves_pair.relative_bearing_target_to_own, ves_pair.dcpa, ves_pair.tcpa
    
    return cri.calculate_cri(own, target, dist_euclid, speed_r, rel_movement_direction, azimuth_angle_target_to_own, relative_bearing_target_to_own, dcpa, tcpa)[-1]

In [20]:
%%timeit -n 100
timeslice_max_traffic.apply(calculate_cri_, axis=1)

2.3 ms ± 47.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
